In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
from dotenv import load_dotenv
load_dotenv()


True

In [5]:
apiKey = os.getenv("OPENAI_API_KEY") 

In [7]:
llm = ChatOpenAI(openai_api_key=apiKey,model_name="gpt-3.5-turbo",temperature=0.5)

/var/folders/tl/2rzqd82j4xncc4bmthg5hlbr0000gn/T/ipykernel_5461/749673684.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=apiKey,model_name="gpt-3.5-turbo",temperature=0.5)


In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
 "1" : {
    "mcq": "multiple choice question",
    "options": {
        "a": "choice here",
        "b": "choice here",
        "c": "choice here",
        "d": "choice here",
},
"correct": "correct answer",
},
 "2" : {
    "mcq": "multiple choice question",
    "options": {
        "a": "choice here",
        "b": "choice here",
        "c": "choice here",
        "d": "choice here",
},
"correct": "correct answer",
},
 "3" : {
    "mcq": "multiple choice question",
    "options": {
        "a": "choice here",
        "b": "choice here",
        "c": "choice here",
        "d": "choice here",
},
"correct": "correct answer",
},
}

In [10]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQS
### RESPONSE_JSON
{response_json}
"""

In [11]:
quiz_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [12]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_prompt,output_key="quiz",verbose=True)

/var/folders/tl/2rzqd82j4xncc4bmthg5hlbr0000gn/T/ipykernel_5461/1380526992.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm=llm,prompt=quiz_prompt,output_key="quiz",verbose=True)


In [13]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis if the quiz is not at per with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}
Check from an expert English Writer of the above quiz:" \
"""

In [14]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
    )

In [15]:
review_chain = LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)


In [16]:
file_path = r"/Users/mac/Desktop/MCQGEN/data.txt"

In [17]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],output_variables=["quiz","review"],verbose=True)

In [18]:
with open(file_path,'r') as file:
    Text = file.read()

In [19]:
Number = 5
Subject = "Jaya pataka Swami"
Tone = "simple"

In [20]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text":Text,
            "number":Number,
            "subject":Subject,
            "tone":Tone,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )

/var/folders/tl/2rzqd82j4xncc4bmthg5hlbr0000gn/T/ipykernel_5461/244778231.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Jayapataka Swami was born in Milwaukee, Wisconsin as Gordon John Erdman II to Gordon John Erdman and Lorraine Erdman (Golich).[5] He attended St. John's Northwestern Military Academy and Brown University.[5] There, as a fresh enrollee, he was influenced by a guest lecture on the life of Buddha that he lost all interest in his studies and began searching for a spiritual teacher.[5] After some time of searching he concluded he would have to go to India to find his teacher.[citation needed]
Before leaving for India, Gordon came across some Hare Krishna devotees, doing kirtan and distributing Back To Godhead magazine. Shortly after he visited the San Francisco ISKCON center, where he met Jayananda das who introduced him to the Ratha Yatra festival. His first service in ISKCON was to help build the Ratha Cart. From there he travelled to Montreal, Quebec, Canada where he met Srila Prab

In [21]:
print(f"Total token: {cb.total_tokens}")
print(f"prompt token: {cb.prompt_tokens}")
print(f"Total cost: {cb.total_cost}")

Total token: 1536
prompt token: 1094
Total cost: 0.002525


In [22]:
response

{'text': "Jayapataka Swami was born in Milwaukee, Wisconsin as Gordon John Erdman II to Gordon John Erdman and Lorraine Erdman (Golich).[5] He attended St. John's Northwestern Military Academy and Brown University.[5] There, as a fresh enrollee, he was influenced by a guest lecture on the life of Buddha that he lost all interest in his studies and began searching for a spiritual teacher.[5] After some time of searching he concluded he would have to go to India to find his teacher.[citation needed]\nBefore leaving for India, Gordon came across some Hare Krishna devotees, doing kirtan and distributing Back To Godhead magazine. Shortly after he visited the San Francisco ISKCON center, where he met Jayananda das who introduced him to the Ratha Yatra festival. His first service in ISKCON was to help build the Ratha Cart. From there he travelled to Montreal, Quebec, Canada where he met Srila Prabhupada. John took first initiation in Montreal, Quebec, Canada and was given the name 'Jayapataka

In [23]:
quiz = response.get("quiz")

In [24]:
json.loads(quiz)

{'1': {'mcq': 'Where was Jayapataka Swami born?',
  'options': {'a': 'New York',
   'b': 'Milwaukee',
   'c': 'San Francisco',
   'd': 'Toronto'},
  'correct': 'b'},
 '2': {'mcq': "What was Jayapataka Swami's name before taking initiation in ISKCON?",
  'options': {'a': 'Gordon John Erdman II',
   'b': 'Jayananda das',
   'c': 'Lorraine Erdman',
   'd': 'Golich'},
  'correct': 'a'},
 '3': {'mcq': 'Where did Jayapataka Swami take his first initiation in ISKCON?',
  'options': {'a': 'New York',
   'b': 'Montreal',
   'c': 'Toronto',
   'd': 'San Francisco'},
  'correct': 'b'},
 '4': {'mcq': "What was Jayapataka Swami's first service in ISKCON?",
  'options': {'a': 'Book printing',
   'b': 'Ratha Cart building',
   'c': 'Temple opening',
   'd': 'Kirtan singing'},
  'correct': 'b'},
 '5': {'mcq': 'Who instructed Jayapataka Swami to go to India?',
  'options': {'a': 'Jayananda das',
   'b': 'Gordon John Erdman II',
   'c': 'Srila Prabhupada',
   'd': 'Lorraine Erdman'},
  'correct': 'c'}}